In [ ]:
import ee 
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='ee-ebuehler1996')


Successfully saved authorization token.


In [45]:
import requests
import json
import ee
import pandas as pd

# Initialize the Earth Engine module
ee.Initialize()

# Fetch data from the API
url = "https://opendata.rhein-kreis-neuss.de/api/explore/v2.1/catalog/datasets/nrw-postleitzahlen/records?limit=20&refine=plz_name%3A%22Aachen%22"
response = requests.get(url)
data = response.json()

# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse the response to JSON
else:
    raise Exception(f"Failed to retrieve data: {response.status_code}")

# Function to convert JSON geometry to ee.Geometry
def json_to_ee_geometry(geometry_json):
    coordinates = geometry_json['coordinates'][0]
    return ee.Geometry.Polygon(coordinates)

# Extract geometries and convert to ee.Geometry
plz_geometries = {}
for result in data['results']:
    plz_code = result['name']
    geometry_json = result['geometry']['geometry']
    plz_geometries[plz_code] = json_to_ee_geometry(geometry_json)

# Define a function to calculate average NDVI for a given PLZ polygon in 2020
def calculate_average_ndvi(plz_code, geometry):
    # Define a function to calculate NDVI
    def calculate_ndvi(image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')  # B8 is NIR and B4 is Red
        return image.addBands(ndvi)

    # Filter Sentinel-2 images for 2020 and the given geometry
    filtered_images = ee.ImageCollection('COPERNICUS/S2') \
                        .filterBounds(geometry) \
                        .filterDate('2020-01-01', '2020-12-31') \
                        .map(calculate_ndvi)

    # Select the NDVI band
    ndvi_time_series = filtered_images.select('NDVI')

    # Define a function to calculate mean NDVI for each image over the region of interest
    def mean_ndvi(image):
        mean = image.reduceRegion(
            reducer=ee.Reducer.mean(), 
            geometry=geometry, 
            scale=10
        ).get('NDVI')
        return ee.Feature(None, {
            'mean_ndvi': mean
        })

    # Map the function over the NDVI collection
    mean_ndvi_series = ndvi_time_series.map(mean_ndvi)

    # Calculate the average of the mean NDVI values
    mean_ndvi_result = mean_ndvi_series.reduceColumns(ee.Reducer.mean(), ['mean_ndvi'])

    # Get the mean NDVI value from the result
    mean_ndvi_value = mean_ndvi_result.get('mean')

    # Check if mean_ndvi_value exists
    if mean_ndvi_value is None:
        print(f"Warning: No mean NDVI value found for PLZ {plz_code}.")
        mean_ndvi_value = None
    else:
        mean_ndvi_value = mean_ndvi_value.getInfo()  # Get the result from Earth Engine

    result = {
        'plz_code': plz_code,
        'mean_ndvi': mean_ndvi_value
    }

    return result

# Apply the function to each PLZ and collect the results
results = []
for plz_code, geometry in plz_geometries.items():
    result = calculate_average_ndvi(plz_code, geometry)
    results.append(result)

# Convert the results to a pandas DataFrame
df = pd.DataFrame(results)

# Print the DataFrame
print(df)

# Save to CSV if needed
df.to_csv('ndvi_2020_results.csv', index=False)


  plz_code  mean_ndvi
0    52066   0.207245
1    52074   0.282947
2    52070   0.200926
3    52078   0.230341
4    52072   0.229945
5    52062   0.099158
6    52076   0.283134
7    52064   0.135038
8    52068   0.145891
9    52080   0.239880
